In [2]:
import pandas as pd
import sklearn as ktl

In [13]:
bol = pd.read_csv("/Users/hendrixperalta/Desktop/bolivia/data/GeoDS4Bolivia.csv")
#list(bol.columns)

#'imds'
#"ln_NTLpc2012"


In [14]:
x = bol[["ln_NTLpc2012", 'pop2012', "sdg8_5_oprm", "sdg8_5_ofrm", "sdg1_1_ubn", "sdg3_7_afr"]]
y = bol['imds']